In [1]:
#CODE FOR INITIALIZING REWARD TABLE
import pickle
from pprint import pprint as pp
import datetime
import glob
import random
import os
from textblob import TextBlob
import pandas as pd
import pickle
import os

dir='C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/data/300_convo/'
strategies_dict=pickle.load( open("C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/strategies.pkl", "rb" ) )

N_TURN = 3
list1=[]
bot_strategies=[]
learning_rate=0.01
discount_factor=0.9
epsilon=0.9

user_utterances=[]
strategies_list=[]

evaluation_dict={}
user_lines=[]
#state action table
R_table=pd.DataFrame.from_items([('qs+b', [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("qs+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("qs'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs'e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls+b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                            ('qls-b', [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("q'ls-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls'e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("q'l's+b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                          ("q'l's'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's'e", [0.0, 0.0, 0.0,0.0,0.0,0.0])],
                                            orient='index', 
                                          columns=['question', 'elaborate', 'opinion','affirmative','negative','neutral'])

#temporary dictionary which stores the count of the recorded utterance till date,this is used for calculating the average to be stored in state table
temp_dict={"qs+b":0,"qs+e":0,"qs-b":0,"qs-e":0,"qs'b":0,"qs'e":0,"q'ls+b":0,"q'ls+e":0,"q'ls-b":0,"q'ls-e":0,"q'ls'b":0,"q'ls'e":0,
           "q'l's+b":0,"q'l's+e":0,"q'l's-b":0,"q'l's-e":0,"q'l's'b":0,"q'l's'e":0}

temp_dict1={"qs+b":0,"qs+e":0,"qs-b":0,"qs-e":0,"qs'b":0,"qs'e":0,"q'ls+b":0,"q'ls+e":0,"q'ls-b":0,"q'ls-e":0,"q'ls'b":0,"q'ls'e":0,
           "q'l's+b":0,"q'l's+e":0,"q'l's-b":0,"q'l's-e":0,"q'l's'b":0,"q'l's'e":0}
Q_table=pd.DataFrame.from_items([('qs+b', [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("qs+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("qs'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("qs'e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls+b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                            ('qls-b', [0.0, 0.0, 0.0,0.0,0.0,0.0]), ("q'ls-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'ls'e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                           ("q'l's+b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's+e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's-b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's-e", [0.0, 0.0, 0.0,0.0,0.0,0.0]),
                                          ("q'l's'b", [0.0, 0.0, 0.0,0.0,0.0,0.0]),("q'l's'e", [0.0, 0.0, 0.0,0.0,0.0,0.0])],
                                            orient='index', 
                                          columns=['question', 'elaborate', 'opinion','affirmative','negative','neutral'])

In [2]:
#utility functions
def check_strategy(response):
    for key, value in strategies_dict.items():
        if response in value:
            #print(key)
            break
            
    return key
def check_strategies(response):
    temp_list=response.split(":")
    temp_list2=temp_list[0].split('_')
    return temp_list2[1]
    

def get_utterance(line):
    temp_list=line.split(":")
    value=temp_list[3]
    return value

def populate_evaluation(line):
    list1=line.split(',')
    for values in list1:
        if values is not '':
            list2=values.split('=')
            evaluation_dict[list2[0]]=list2[1]

#utility functions
def next_state(utterance_no):
    if not utterance_no==len(user_utterances)-1:
        return user_utterances[utterance_no+1]
    else:
        return 0
    
def max_q_value(utterance):
    max=0
    for column in Q_table:
        if Q_table.at[utterance,column]>max:
            max=Q_table.at[utterance,column]
    return max

def training(indexes):
    
    for line in user_utterances:
        next_utterance=next_state(indexes)
        prev_value=Q_table.at[user_utterances[indexes],strategies_list[indexes]]
        if next_utterance is not 0:
            Q_table.at[user_utterances[indexes],strategies_list[indexes]]=((temp_dict[user_utterances[indexes]]-1)*Q_table.at[user_utterances[indexes],strategies_list[indexes]]+R_table.at[user_utterances[indexes],strategies_list[indexes]] + discount_factor*max_q_value(next_utterance))/temp_dict[user_utterances[indexes]] 
        else:
            Q_table.at[user_utterances[indexes],strategies_list[indexes]]=R_table.at[user_utterances[indexes],strategies_list[indexes]]

#def training_learning():
    

#def learn():
    

In [3]:
list1=[]
user_utterances=[]
bot_strategies=[]
#function which updates the state value table for each conversation where list1 is the utterance of the user and list2 is the strategy
#applied by the bot
def record_check(list1,list2,evaluated_score):
    i=0
    for key in list1:
        #print state_action_dict.at[key,list2[i]]
        previous_value=state_action_dict.at[key,list2[i]]
        temp_dict[key]=temp_dict[key]+1
        print(float(evaluated_score))
        #print temp_dict[key],((previous_value*(temp_dict[key]-1))+evaluated_score)/temp_dict[key]
        value=((previous_value*(temp_dict[key]-1))+float(evaluated_score))/temp_dict[key]
        print(value)
        state_action_dict.at[key,list2[i]]=value
        i=i+1
    

def rows_utterance(list1):
    
    for values in list1:
        if question_or_not(values)=="q":
            if sentiment(values)=="s+":
                if beginning_end(values,list1)=="b":
                    result="qs+b"
                else:
                    result="qs-e"
            elif sentiment(values)=="s-":
                if beginning_end(values,list1)=="b":
                    result="qs-b"
                else:
                    result="qs-e"
            else:
                if beginning_end(values,list1)=="b":
                    result="qs'b"
                else:
                    result="qs'e"
        else:
            if count_no_words(values)=="l":
            
                if sentiment(values)=="s+":
                
                    if beginning_end(values,list1)=="b":
                        result="q'ls+b"
                    else:
                        result="q'ls+e"
                elif sentiment(values)=="s-":
                    if beginning_end(values,list1)=="b":
                        result="q'ls-b"
                    else:
                        result="q'ls-e"
                else:
                    if beginning_end(values,list1)=="b":
                        result="q'ls'b"
                    else:
                        result="q'ls'e"
            else:
                if sentiment(values)=="s+":
                
                    if beginning_end(values,list1)=="b":
                        result="q'l's+b"
                    else:
                        result="q'l's+e"
                elif sentiment(values)=="s-":
                    if beginning_end(values,list1)=="b":
                        result="q'l's-b"
                    else:
                        result="q'l's-e"
                else:
                    if beginning_end(values,list1)=="b":
                        result="q'l's'b"
                    else:
                        result="q'l's'e"
        user_utterances.append(result)
    return user_utterances

def question_or_not(words):
    if "?" not in words:
        q="q'"
    else:
        q="q"
    #print q
    return q

def count_no_words(words):
    no=len(words.split())
    if no>=10:
        #print "long"
        l="l"
    else:
        #print "short"
        l="l'"
    return l

def sentiment(words):
    blob = TextBlob(words)
    senti=blob.sentiment
    if senti.polarity<0.2 and senti.polarity>-0.2:
        s="s'"
    else:
        if senti.polarity>0:
            s="s+"
        else:
            s="s-"
    return s
    


def beginning_end(line,list):
    length=len(list)
    if list.index(line)==0:
        #print "beginning"
        var="b"
    else:
        #print "ending"
        var="e"
    return var

In [4]:
#CODE FOR EXTRACTING VALUES FROM REWARD TABLE AND LEARNING FROM IT

user_utterances=[]

for filename in os.listdir(dir):
    print("hello")
    file = open('C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/data/300_convo/'+filename,'r')
    strategies_list=[]
    user_lines=[]
    user_utterances=[]
    for lines in file:
        print(lines)
        if not lines.startswith(("overall","engaing","return","interupt","start")):
            if lines.startswith('Bot'):
                strategies_list.insert(len(strategies_list),check_strategies(lines))
            else:
                user_lines.append(get_utterance(lines))
                
        else:
            
            if lines.endswith(','):
                populate_evaluation(lines)
                user_utterances=rows_utterance(user_lines)
                temp_dict[user_utterances[0]]=temp_dict[user_utterances[0]]+1
                R_table.at[user_utterances[0],strategies_list[0]]=((temp_dict[user_utterances[0]]-1)*R_table.at[user_utterances[0],strategies_list[0]]+0.8*float(evaluation_dict['start'])+0.2*float(evaluation_dict['overall']))/temp_dict[user_utterances[0]]
                count=1
                for line in user_utterances[1:len(user_utterances)]:
                    #print(count)
                    temp_dict[user_utterances[count]]=temp_dict[user_utterances[count]]+1
                    R_table.at[user_utterances[count],strategies_list[count]]=((temp_dict[user_utterances[count]]-1)*R_table.at[user_utterances[count],strategies_list[count]]+0.4*float(evaluation_dict['engaing'])+0.4*float(evaluation_dict['interupt'])+0.2*float(evaluation_dict['overall']))/temp_dict[user_utterances[count]]               
                    
                    
                    count=count+1
            else:
                pass
        
    file.close()

hello
Bot_question_23:52:05:What plot would you want to happen differently?

abc_None_23:55:54:Not sure

Bot_elaborate_23:55:54:What?

abc_None_23:56:20:not sure what plot I want to happen differently

Bot_affirmative_23:56:20:Awesome

abc_None_23:56:33:Thanks!

Bot_affirmative_23:56:33:That's neat!

abc_None_23:56:49:alright!

Bot_opinion_23:56:49:Life is not a song, sweetling. Someday you may learn that, to your sorrow.

abc_None_00:00:23:May the force be with you

Bot_neutral_00:00:23:haha

abc_None_00:00:30:hohoho

Bot_question_00:00:30:What is true happiness?

abc_None_00:02:51:Bot becomes more smart

overall=4,start=4,interupt=4,engaing=4,return=4,
hello
Bot_neutral_22:02:05:I need to meditate on that

alec zhang_None_22:02:32:ok...

Bot_opinion_22:02:32:Any man who must say 'I am the king' is no true king.

alec zhang_None_22:03:01:agree, all people who said that died

Bot_neutral_22:03:01:Well okay.

alec zhang_None_22:03:05:okay

Bot_opinion_22:03:05:Life is not a song, sweetl

KeyError: "q'ls-b"

In [5]:
R_table

,question,elaborate,opinion,affirmative,negative,neutral
qs+b,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000
qs+e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
qs-b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
qs-e,0.050000,1.000000,0.290000,0.613333,0.483333,1.742857
qs'b,0.000000,0.056078,0.069565,0.040303,0.044733,0.010526
qs'e,0.025524,0.079129,0.321218,0.082723,1.197930,0.546884
q'ls+b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
q'ls+e,0.000000,0.000000,0.700000,0.000000,0.800000,0.400000
qls-b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
q'ls-e,1.133333,0.000000,0.000000,0.000000,0.000000,0.250000


In [6]:
#for training from the model
for filename in os.listdir(dir):
    print("hello")
    file = open('C://Users/Nehal/Desktop/cmps296a/Reinforcement_learning_NLP/data/300_convo/'+filename,'r')
    strategies_list=[]
    user_lines=[]
    user_utterances=[]
    for lines in file:
        print(lines)
        if not lines.startswith(("overall","engaing","return","interupt","start")):
            if lines.startswith('Bot'):
                strategies_list.insert(len(strategies_list),check_strategies(lines))
            else:
                user_lines.append(get_utterance(lines))
    user_utterances=rows_utterance(user_lines)
    for values in user_utterances:
        indexes=user_utterances.index(values)
        training(indexes)

hello
Bot_question_23:52:05:What plot would you want to happen differently?

abc_None_23:55:54:Not sure

Bot_elaborate_23:55:54:What?

abc_None_23:56:20:not sure what plot I want to happen differently

Bot_affirmative_23:56:20:Awesome

abc_None_23:56:33:Thanks!

Bot_affirmative_23:56:33:That's neat!

abc_None_23:56:49:alright!

Bot_opinion_23:56:49:Life is not a song, sweetling. Someday you may learn that, to your sorrow.

abc_None_00:00:23:May the force be with you

Bot_neutral_00:00:23:haha

abc_None_00:00:30:hohoho

Bot_question_00:00:30:What is true happiness?

abc_None_00:02:51:Bot becomes more smart

overall=4,start=4,interupt=4,engaing=4,return=4,
hello
Bot_neutral_22:02:05:I need to meditate on that

alec zhang_None_22:02:32:ok...

Bot_opinion_22:02:32:Any man who must say 'I am the king' is no true king.

alec zhang_None_22:03:01:agree, all people who said that died

Bot_neutral_22:03:01:Well okay.

alec zhang_None_22:03:05:okay

Bot_opinion_22:03:05:Life is not a song, sweetl

KeyError: "q'ls-b"

In [7]:
Q_table

,question,elaborate,opinion,affirmative,negative,neutral
qs+b,0.000000,1.813319,0.000000,0.000000,0.000000,2.789662
qs+e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
qs-b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
qs-e,0.915313,3.437755,1.652571,1.534513,0.639147,2.500078
qs'b,0.000000,2.406756,1.376736,1.720442,1.763716,0.962461
qs'e,0.640552,1.544994,1.128430,0.997417,1.485191,0.546884
q'ls+b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
q'ls+e,0.000000,0.000000,2.846490,0.000000,2.306902,3.257771
qls-b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
q'ls-e,3.267731,0.000000,1.439553,0.000000,0.000000,2.565829
